# Projecting images to latent space with StyleGAN2.


## Requirements

### Switch to Tensorflow 1.x

The default TensorFlow version in Colab switched from  1.x to 2.x on the 27th of March, 2020.

Reference: https://colab.research.google.com/notebooks/tensorflow_version.ipynb

We switch to version 1.x to avoid the following error:
> `ModuleNotFoundError: No module named 'tensorflow.contrib' colab`

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install my fork of StyleGAN2

Reference: https://github.com/richard1634/stylegan2


Clone my fork.

In [2]:
%cd /content/

/content


In [3]:
%rm -rf stylegan2/
!git clone https://github.com/richard1634/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 63 (delta 6), reused 63 (delta 6), pack-reused 0
Unpacking objects: 100% (63/63), done.


Install.

In [4]:
!nvcc test_nvcc.cu -o test_nvcc -run

gcc: error: test_nvcc.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.


### Mount Google Drive

In [5]:
!pip install Google-Colab-Transfer

In [6]:
import colab_transfer

In [7]:
colab_transfer.mount_google_drive()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
%cd /content/stylegan2/

/content/stylegan2


##### Import Images from google drive, I put it my .png files in a folder, tar.gz'ed them and uploaded them to drive. The faces in my images are aligned.



In [13]:
archive_name = 'faces.tar.gz'  # the name of my folder on drive. It's a folder with .png files of aligned faces.
colab_transfer.copy_file(file_name=archive_name,
                         destination=colab_transfer.get_path_to_home_of_local_machine() + 'stylegan2/')

Copying /content/drive/My Drive/faces.tar.gz to /content/stylegan2/faces.tar.gz


In [ ]:
!du -sh $archive_name
!tar -xvf $archive_name

### Projecting images to latent space

Reference: https://github.com/woctezuma/stylegan2/tree/tiled-projector

#### `--no-tiled` projection

This is the projector suggested in [`rolux/stylegan2encoder`](https://github.com/rolux/stylegan2encoder), using `W(18, 512)`.

Visual fidelity is higher, but semantic fidelity is lower.

In [16]:
#%cd /content/stylegan2/
!pwd

/content/stylegan2



###### Vanilla

In [15]:
import os
archive_name = "faces"
src_files = sorted([os.path.join(archive_name, f) for f in os.listdir(archive_name) if f[0] not in '._'])
print(len(src_files))  # number of images
print(src_files[0])    # example of whats in the folder

996
faces/face-1.png


###Projections are done in batches of 100 to reduce using all of colabs avaliable memory

In [ ]:
!python project_images.py faces/ generated_images_no_tiled 0 100 --no-tiled --num-steps=50   #project images to latent space, starting at image 0 and going to 100, step 50 times. 
!python project_images.py faces/ generated_images_no_tiled 100 200 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 200 300 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 300 400 --no-tiled --num-steps=50

In [ ]:

!python project_images.py faces/ generated_images_no_tiled 400 500 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 500 600 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 600 700 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 700 800 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 800 900 --no-tiled --num-steps=50
!python project_images.py faces/ generated_images_no_tiled 900 996 --no-tiled --num-steps=50

##### Export projected images

In [ ]:
archive_name = 'faces_latent.tar.gz'  # i export it as faces_latent.tar.gz to my google drive.

In [ ]:
folder_name = 'generated_images_no_tiled'

!tar -cvf $archive_name $folder_name/
!du -sh $archive_name

In [ ]:
colab_transfer.copy_file(file_name=archive_name,
                         source=colab_transfer.get_path_to_home_of_local_machine() + 'stylegan2/',
                         destination=colab_transfer.get_path_to_home_of_google_drive())

File /content/drive/My Drive/faces_latent.tar.gz already exists. Copy skipped.


#### We've finished creating latent vectors from the images. Next we'll have to hand classify the images into those with glasses vs those with no glasses to help train our classifier. Move on to https://colab.research.google.com/github/richard1634/Latent-Space-Boundary-Trainer-for-StyleGan2/blob/master/Glasses_classifier.ipynb